In [1]:
from rubiks import *
from solver import *
from heuristics import *
import timeit
import pandas as pd
import pickle

In [2]:
h = {
    "numberOfCorrectColorsOnFace": numberOfCorrectColorsOnFace,
    "numberOfPairedCornerNEdges": numberOfPairedCornerNEdges,
    "oneStepCornerEdgePair": oneStepCornerEdgePair,
    "compound1": compound1,
    "compound2": compound2,
    "numberOfSolidRows": numberOfSolidRows,
    "numberOfSolidFaces": numberOfSolidFaces,
    "compound3": numberOfSolidRowsWithSolidFace
}

In [3]:
results = {}

In [4]:
# unload scrambles
filename = 'testScrambles.pkl'

with open(filename, "rb") as fp:   # Unpickling
    scrambles = pickle.load(fp)

In [13]:
all_steps = [3,5,7,10]
# all_steps = [3,5]
eps = 100
scrambles = {steps: {} for steps in all_steps}
# make scrambles
for steps in all_steps:
    for i in range(eps):
        scramble = generateScramble(steps)
        scrambles[steps][scramble] = None
    

In [14]:
results  = {name: {} for name in h}
for (name, heuristic) in h.items():
    result = {}
    for step, step_scrambles in scrambles.items():
        time = 0
        solved = 0
        for scramble in step_scrambles:
            c = cube()
            c.move(scramble)

            s = solver(c, h=heuristic)
            start = timeit.default_timer()
            s.solve(maxDepth=10, display=False, verbose=False, earlyTermination=500)
            if s.solutionFound:
                solved +=1
            stop = timeit.default_timer()
            timeTaken = stop - start
            time += timeTaken

        avg_time = time/len(step_scrambles)
        result['{}_steps_avg_time'.format(step)] = avg_time
        result['{}_steps_solves (%)'.format(step)] = (solved/len(step_scrambles)) * 100
        results[name] = result
        

In [15]:
pd.DataFrame.from_dict(results, orient='index')

,3_steps_avg_time,3_steps_solves (%),5_steps_avg_time,5_steps_solves (%),7_steps_avg_time,7_steps_solves (%),10_steps_avg_time,10_steps_solves (%)
numberOfCorrectColorsOnFace,0.002104,93.814433,0.084257,42.0,0.138181,10.0,0.180338,0.0
numberOfPairedCornerNEdges,0.003432,88.659794,0.129583,50.0,0.267979,23.0,0.764959,0.0
oneStepCornerEdgePair,4.228778,48.453608,6.637177,19.0,10.176654,4.0,5.477501,0.0
compound1,0.005723,79.381443,0.150988,47.0,0.421652,22.0,0.513609,3.0
compound2,0.004554,82.474227,0.106375,52.0,0.405153,26.0,0.498771,5.0
numberOfSolidRows,0.003281,97.938144,0.040403,72.0,0.135811,53.0,0.504943,6.0
numberOfSolidFaces,0.002615,100.000000,0.070067,67.0,0.165118,26.0,0.234538,1.0
compound3,0.003963,97.938144,0.031448,72.0,0.164770,48.0,0.392717,7.0
